In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import torchvision
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
# features = train_data.drop(columns=['Survived'])
# f = features.values
# f[0,:]
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
def get_parameters(df):
    parameters = {}
    for col in df.columns[df.isnull().any()]:
        if df[col].dtype == 'float64' or df[col].dtype == 'int64':
            strategy = 'mean'
        else:
            strategy = 'most_frequent'
        
        missing_values = df[col][df[col].isnull()].values[0]
        parameters[col] = {'missing_values' : missing_values, 'strategy' : strategy}
    
    return parameters
 
get_parameters(train_data)

{'Age': {'missing_values': nan, 'strategy': 'mean'},
 'Cabin': {'missing_values': nan, 'strategy': 'most_frequent'},
 'Embarked': {'missing_values': nan, 'strategy': 'most_frequent'}}

In [4]:
parameters = get_parameters(train_data)

In [5]:
for col, param in parameters.items():
    missing_values = param['missing_values']
    strategy = param['strategy']
    imp = SimpleImputer(strategy = strategy, missing_values = missing_values)
    train_data[col] = imp.fit_transform(train_data[[col]]).ravel()
for col, param in parameters.items():
    missing_values = param['missing_values']
    strategy = param['strategy']
    imp = SimpleImputer(strategy = strategy, missing_values = missing_values)
    test_data[col] = imp.fit_transform(test_data[[col]]).ravel()

In [6]:
train_data['Sex'] = OneHotEncoder().fit_transform(train_data[['Sex']]).toarray()[:,1]
encoder =  LabelEncoder()
train_data['Embarked'] = encoder.fit_transform(train_data['Embarked'])
test_data['Sex'] = OneHotEncoder().fit_transform(test_data[['Sex']]).toarray()[:,1]
test_data['Embarked'] = encoder.fit_transform(test_data['Embarked'])
train_data
test_data



,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1.0,34.50000,0,0,330911,7.8292,B57 B59 B63 B66,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0.0,47.00000,1,0,363272,7.0000,B57 B59 B63 B66,2
2,894,2,"Myles, Mr. Thomas Francis",1.0,62.00000,0,0,240276,9.6875,B57 B59 B63 B66,1
3,895,3,"Wirz, Mr. Albert",1.0,27.00000,0,0,315154,8.6625,B57 B59 B63 B66,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0.0,22.00000,1,1,3101298,12.2875,B57 B59 B63 B66,2
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1.0,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,2
414,1306,1,"Oliva y Ocana, Dona. Fermina",0.0,39.00000,0,0,PC 17758,108.9000,C105,0
415,1307,3,"Saether, Mr. Simon Sivertsen",1.0,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,2
416,1308,3,"Ware, Mr. Frederick",1.0,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,2


In [7]:
X = torch.tensor(train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].values, dtype=torch.float)
X_test = torch.tensor(test_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].values, dtype=torch.float)

In [8]:
Y = torch.tensor(train_data[['Survived']].values, dtype=torch.float)

In [9]:
#this is now the training part

input_size = 7
hidden_size = 100
num_classes = 1
num_epochs = 10
batch_size = 5
learning_rate = 0.0001



In [10]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.sigmoid(out)
        
        return out
    

In [11]:
model = NeuralNet(input_size, hidden_size, num_classes)


In [12]:
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [14]:
dataset = CustomDataset(X, Y)
total_steps = len(dataset)
# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [15]:
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i , (sample, targets) in enumerate(dataloader):
        y_pred = model.forward(sample)
        
        loss = criterion(y_pred, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        predicted = (y_pred > 0.5).long()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
        if (i+1) % 5 == 0:
            accuracy = correct / total * 100
            print(f'epoch {epoch+1} / {num_epochs} ,step {i+1}/{total_steps}, accuracy = {accuracy} ')
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{total_steps}, loss = {loss.item()}')
        

epoch 1 / 10 ,step 5/891, accuracy = 36.0 
epoch 1 / 10, step 5/891, loss = 4.954632759094238
epoch 1 / 10 ,step 10/891, accuracy = 38.0 
epoch 1 / 10, step 10/891, loss = 5.4951043128967285
epoch 1 / 10 ,step 15/891, accuracy = 38.666666666666664 
epoch 1 / 10, step 15/891, loss = 3.0808446407318115
epoch 1 / 10 ,step 20/891, accuracy = 39.0 
epoch 1 / 10, step 20/891, loss = 2.097285747528076
epoch 1 / 10 ,step 25/891, accuracy = 40.0 
epoch 1 / 10, step 25/891, loss = 3.710951566696167
epoch 1 / 10 ,step 30/891, accuracy = 37.333333333333336 
epoch 1 / 10, step 30/891, loss = 2.738940715789795
epoch 1 / 10 ,step 35/891, accuracy = 36.57142857142857 
epoch 1 / 10, step 35/891, loss = 2.5213825702667236
epoch 1 / 10 ,step 40/891, accuracy = 37.0 
epoch 1 / 10, step 40/891, loss = 3.198112726211548
epoch 1 / 10 ,step 45/891, accuracy = 38.22222222222222 
epoch 1 / 10, step 45/891, loss = 3.1429991722106934
epoch 1 / 10 ,step 50/891, accuracy = 37.2 
epoch 1 / 10, step 50/891, loss = 4.